# Topple the Target
#### Zain Naqavi

## Overview
This code will create a game resembling 'Angry Birds' where the goal is to topple a target with a projectile by inputting the launch angle and velocity.

The program will: 
* Calculate the path of the projectile.
* Calculate the momentum and impulse transferred by the projectile to the target upon impact.
* Calculate the torque on the target from the collision.
* Determine if the target will topple if the restoring torque is less than the applied torque.

The code consists of nested if and while structures which determine if a collision between the projectile and target has occured. If the condition of a collision is met, the code will determine whether the target will topple using the relevant calculations outlined below.

The basic algorithm structure that the 'game' part of the code follows is outlined below: 
* Ask the user to input the angle and launch velocity of the projectile.
* This will start the projectile animation until either the target or the ground is hit. 
* If the projectile missed the target, the game will restart. 
* If the projectile hit the target, the calculations of momentum, impulse and torque will be carried out to determine if the target will topple. 
* If the applied torque caused by the impact of the collision is not greater than the restoring torque of the target, the target will not topple and the game will restart. 
* If the applied torque is enough to cause the target to topple, the game will end. 

## Theory

### Projectile motion

The motion of the ball is modelled neglecting air resistance. Given the initial speed $v_0$ and the launch angle $\theta$, the position and momentum of a projectile can be calculated.


The position $\mathbf{r}$ of the projectile at any time $t$ is given by:

$$ r_x = x_0 + v_0 t \cos\theta $$
$$ r_y = y_0 + v_0 t \sin\theta − \frac{1}{2}g t^2 $$


The momentum $\mathbf{p}$ of the projectile is given by:

$$ p_x = m v_0 \cos\theta $$
$$ p_y = m v_0 \sin\theta − m g t $$

### Collisions

The collision between the ball and target is modelled as elastic and the ball is assumed to transfer all momentum to the target. The model assumes the center of mass of the target is at its geometric center and its center of rotation is at the bottom right.


The restoring torque of the target is:

$$ \boldsymbol{\tau}_{restoring} = \mathbf{F}_{gravity} \times \mathbf{d}_r $$

where $\mathbf{F}_{gravity} = m \mathbf{g}$ is the force due to gravity acting through the center of mass and $\mathbf{d}_r$ is the horizontal distance  between the point of rotation (bottom right corner) and the centre of mass.


The torque applied by the collision is:

$$ \boldsymbol{\tau}_{applied} = \mathbf{F}_{applied} \times \mathbf{d}_a $$

where $\mathbf{F}_{applied} = \Delta\mathbf{p} / \Delta t$ is the force transferred during the finite impact time by the momentum of the ball on impact and $\mathbf{d}_a$ is the vector from the point of rotation (bottom right-hand corner) to the point of impact.


The target will topple if:

$$|\boldsymbol{\tau}_{applied}| > |\boldsymbol{\tau}_{restoring}|$$

## Code

In [1]:
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow, text

# Set up the scene
scene = canvas(width = 640, height = 480, center = vector(8,5,0), range = 8)
ground = curve(pos = [(0,0,0), (16,0,0)], color = color.green)

dt = 0.001  # animation time step, s
g = 9.81    # gravitational acceleration, m/s^2

# Target initial conditions (mass, dimensions, position)
m_t = 100                # mass of target, kg
w_t = 0.5                # width of target, m
h_t = 2.0                # height of target, m
x0_t = 10*random() + 5   # initial target centre-of-mass x-coordinate (random value between 5m and 15m)
y0_t = h_t / 2           # initial target centre-of-mass y-coordinate, m

# Platform initial conditions 
h_p = random()   # height of platform (random value between 0m and 1m)
x0_p = 0.0       # initial platform centre-of-mass x-coordinate, m
y0_p = h_p / 2   # initial platform centre-of-mass y-coordinate, m

# Ball initial conditions (position, angle, speed, momentum)
m_b = 0.1    # mass of ball, kg
r_b = 0.05   # radius of ball, m
x0_b = 0.0   # initial ball x-coordinate, m
y0_b = h_p   # initial ball y-coordinate, m

# Set up objects
target = box(pos = vector(x0_t,y0_t,0), length = w_t, height = h_t, width = 0.5, color = color.orange)
platform = box(pos = vector(x0_p,y0_p,0), length = 1, height = h_p, width = 1, color = color.white)
ball = sphere(pos = vector(x0_b,y0_b,0), radius = 0.3, color = color.red)
momentum = arrow(pos = ball.pos, axis = vector(0,0,0), shaftwidth = 0.1, color = color.magenta)

# labels
l_attempt = label(pos = vector(1,-1,0))

# Start game
topple = False          # boolean variable for target topple
attempt = 0             # initialise number of user attempts
while topple == False:  # loop runs until the target is toppled
    attempt +=1         # increase attempt number
    
    # user input
    print("\nAttempt #", attempt)
    dtheta_b = float(input("Input the initial angle (in degrees) to launch ball: "))
    theta_b = np.radians(dtheta_b)
    v0_b = float(input("Input the initial speed (in metres/second) to launch ball: "))
    
    # update label
    l_attempt.text = 'Attempt #{0:d}\nLaunch angle: {1:0.1f} deg\nLaunch velocity: {2:0.1f} m/s^2'.format(attempt,dtheta_b,v0_b)
    
    # update ball initial conditions
    x0_b = 0.0                       # initial ball x-coordinate, m
    y0_b = h_p                       # initial ball y-coordinate, m (always launched from same height)
    ball.pos = vector(x0_b,y0_b,0)   # initial position of ball
    px_b = m_b*v0_b*np.cos(theta_b)  # initial ball momentum, Ns -in x-direction
    py_b = m_b*v0_b*np.sin(theta_b)  # initial ball momentum, Ns -in y-direction
    
    # Start the projectile animation
    t = 0                       # initial time, s
    collision = False           # boolean variable for collision between ball and target
    while collision == False:   # loop runs until a collision occurs
        rate(1000)              # restricts animation to 1000 updates per second (since dt=0.001)
        
        if ball.pos.y-r_b > 0:  # position and momentum of ball only updated if ball is above ground
            t = t+dt            # increment value of time to calculate new position and momentum
    
            # equations of motion of ball
            ball.pos.x = x0_b + v0_b*t*np.cos(theta_b)               # new x-position of ball at time t
            ball.pos.y = y0_b + v0_b*t*np.sin(theta_b) - 0.5*g*t**2  # new y-position of ball at time t
            py_b = m_b*v0_b*np.sin(theta_b) - m_b*g*t                # new vertical momentum of ball at time t

            # update arrow representing momentum of ball
            momentum.pos = ball.pos
            momentum.axis = vector(px_b,py_b,0)
        
            # check for collision in each iteration of while loop
            if abs(ball.pos.x - target.pos.x) <= r_b + 0.5 * w_t and ball.pos.y - h_t <= r_b:
                collision = True
                h_c = ball.pos.y               # height of collision, m
                p_c = vector(momentum.axis)    # momentum transferred in collision, Ns
                momentum.axis = vector(0,0,0)  # remove vector representing momentum of ball
                
                if h_c < h_t:  # ball falls only if it does not collide at top of target
                    
                    # Start ball free-fall animation (since all momentum transferred to target)
                    t=0
                    while ball.pos.y-r_b > 0:  # loop runs until only if ball is above ground
                        rate(1000)
                        t = t+dt
                        ball.pos.y = h_c - 0.5*g*t**2  # free-fall under gravity (no initial velocity)
                        
        else:
            print("\nTry again.\n")
            break
            
    # Determine collision properties
    if collision == True:
        d_a = vector((ball.pos.x+r_b)-(x0_t+0.5*w_t),h_c,0)  # vector from the point of rotation to the point of impact on target, m
        F = p_c/0.01                                         # force applied to target by ball, N (contact time is 0.01)
        torque_a = cross(F,d_a)                              # applied torque on target, N (vector)
        torque_r = m_t*g*w_t/2                               # restoring torque of target, N (magnitude) 
        
        # check if target will topple
        if mag(torque_a) > torque_r:
            topple = True
            
            # Start the topple animation
            dphi_t = 0.01           # change in angle in time dt (from w=dphi/dt)
            phi_t = 0               # initial angular displacement, rad
            while phi_t < np.pi/2:  # loop runs until the target has rotated pi/2 radians
                rate(100)           # rate set to 100 since dphi = 0.01
                phi_t=phi_t+dphi_t  # increment value of angle rotated
                target.rotate(angle=-dphi_t, axis=vector(0,0,1), origin=vector(x0_t+w_t/2,0,0)) 
            
            # win text for topple
            text(pos=vector(8,5,0), text='Target Toppled!', align='center', color=color.yellow)
            
            # output collision properties
            print('\nCollision!\nHeight of impact: {0:0.2f} m\nMomentum of ball on impact: {1:0.1f} Ns\nApplied torque: {2:0.0f} N\nRestoring torque: {3:0.0f} N'.format(h_c,mag(p_c),mag(torque_a),torque_r))
            print("\nWell done! The target toppled.")
            
        else:
            # output collision properties
            print('\nCollision!\nHeight of impact: {0:0.2f} m\nMomentum of ball on impact: {1:0.1f} Ns\nApplied torque: {2:0.0f} N\nRestoring torque: {3:0.0f} N'.format(h_c,mag(p_c),mag(torque_a),torque_r))
            print("\nThe target did not topple. Try again.\n")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Attempt # 1
Input the initial angle (in degrees) to launch ball: 80
Input the initial speed (in metres/second) to launch ball: 5

Try again.


Attempt # 2
Input the initial angle (in degrees) to launch ball: 60
Input the initial speed (in metres/second) to launch ball: 10

Try again.


Attempt # 3
Input the initial angle (in degrees) to launch ball: 10
Input the initial speed (in metres/second) to launch ball: 30

Collision!
Height of impact: 1.11 m
Momentum of ball on impact: 3.0 Ns
Applied torque: 345 N
Restoring torque: 245 N

Well done! The target toppled.
